### Project Hack My Ride – Data Preprocessing
**Group members:**
* Andrieux, Nicolas
* Gjini, Jurgen
* Hioco, Tomasso
* Pypaert, Virgile

As the project contains many different data sources and that some treatments are very similar between the questions, we decided to provide a Jupyter Notebook per question we answered to for the sake of clarity.

The first part, in this file, contains all the data preprocessing we made on the datasets used in the various questions :
- Vehicle positions ;
- GTFS files ;
- Shapefiles ;
- GPS tracks by Mahmoud and Jean-Philippe

#### Python packages import
Here below are all the libraries that are necessary to the data preprocessing from cleaning to representation.

In [6]:
import os
import json
import pyproj
import pandas as pd
import geopandas as gpd
import gtfs_functions as gtfs
import numpy as np
import string

from zipfile import ZipFile
from shapely.geometry import LineString
from shapely.ops import transform
from functools import partial

import matplotlib.pyplot as plt

In [3]:
# Disable warnings about future superseded functions
import warnings
warnings.filterwarnings('ignore')

#### Working directories
As we all have different folder structure, you can provide below the path to the various files we are using in this notebook.

In [4]:
vehiclePosition_folder = 'data/'
gtfs_file = 'gtfs23Sept.zip'
gpstracks_file = 'GPStracks.csv'
shapefile_stops = 'shapefiles/ACTU_STOPS.shp'
shapefile_lines = 'shapefiles/ACTU_LINES.shp'

# Folder to export CSV when they have been treated and temporary folder when extracting ZIP files
export_folder = 'export/'
temporary_folder = 'temp/'

### Data preprocessing
#### Vehicle positions
The first operation consists in flattening and merging the files in order to obtain a single 5-columns Pandas dataframe. This is done using the functions below.

In [4]:
# Build the list of JSON files contained in the current folder
fileList = []
for file in os.listdir(vehiclePosition_folder):
    if file.endswith('.json'):
        fileList.append(vehiclePosition_folder + file)

In [6]:
positionsList = []

# Load the content from all the files
for currentFile in fileList:
    print(f'Processing {currentFile}')
    # Read the file
    f = open(currentFile)
    data = json.load(f)
    f.close()

    # Process the content of the file
    for d in data['data']:
        for line in d['Responses']:
            if line is not None:
                for l in line['lines']:
                    for pos in l['vehiclePositions']:
                        d_time = d['time']
                        l_lineId = l['lineId']
                        pos_directionId = pos['directionId']
                        pos_pointId = pos['pointId']
                        pos_distanceFromPoint = pos['distanceFromPoint']
                        
                        positionsList.append([d_time, l_lineId, pos_directionId, pos_pointId, pos_distanceFromPoint])

# Merge all subdataframes into a single Pandas DF
positions_df = pd.DataFrame(positionsList, columns=['time', 'lineId', 'directionId', 'pointId', 'distanceFromPoint'])

# Divide the timestamp by 1000 as it is counted in ms
positions_df = positions_df.astype({'time': 'int64'})
positions_df['time'] = positions_df['time'].floordiv(1000)

print(f'Finished !')

# Delete the collection object to free RAM
del positionsList

Processing data/vehiclePosition01.json
Processing data/vehiclePosition02.json
Processing data/vehiclePosition03.json
Processing data/vehiclePosition04.json
Processing data/vehiclePosition05.json
Processing data/vehiclePosition06.json
Processing data/vehiclePosition07.json
Processing data/vehiclePosition08.json
Processing data/vehiclePosition09.json
Processing data/vehiclePosition10.json
Processing data/vehiclePosition11.json
Processing data/vehiclePosition12.json
Processing data/vehiclePosition13.json
Finished !


In [20]:
# Keep a copy of the merged vehicle position dataframe
#positions_df.to_csv(export_folder + 'mergedVehiclePosition.csv', index=False)
# Load a copy of the merged vehicle position dataframe to avoid reprocessing each time
positions_df = pd.read_csv(export_folder + 'mergedVehiclePosition.csv')

#### Line segments
In analogy to graph theory, we can define a bus line as a graph with each stop being a node and each segment representing an edge between a couple of stops.

We use the segments in order to obtain the distance between the different stops to calculate the average speed afterwards.

In [8]:
# All the information stored in the GTFS file
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs('gtfs3Sept.zip')

In [16]:
# Build a Geodataframe for all the segments obtained from GTFS file
segments_gdf = gtfs.cut_gtfs(stop_times, stops, shapes)

# Extract information from other columns
segments_gdf['Code_Ligne'] = segments_gdf['shape_id'].str[:4]
segments_gdf['lineId'] = segments_gdf['shape_id'].str[:3]
segments_gdf['mode'] = segments_gdf['shape_id'].str[3:4]
segments_gdf['mode'] = segments_gdf['mode'].str.upper()

segments_gdf['start_stop_id'] = segments_gdf['start_stop_id'].str[:4]
segments_gdf['end_stop_id'] = segments_gdf['end_stop_id'].str[:4]

# Rename columns
segments_gdf = segments_gdf.rename(columns={"direction_id": "Variante", "stop_sequence": "succession"})

# Cast the type
segments_gdf = segments_gdf.astype({'lineId': 'int', 'start_stop_id': 'int', 'end_stop_id': 'int'})
segments_gdf = segments_gdf.round({'distance_m' : 2})

# Select columns
segments_gdf = segments_gdf[["Code_Ligne", "lineId", "mode", "Variante", "succession", "start_stop_id", "end_stop_id", "distance_m"]]

In [15]:
segments_gdf = gtfs.cut_gtfs(stop_times, stops, shapes)
segments_gdf.head()


,route_id,direction_id,stop_sequence,start_stop_name,end_stop_name,start_stop_id,end_stop_id,segment_id,shape_id,geometry,distance_m
0,2,1,1,STOCKEL,CRAINHEM,8161,8151,8161-8151,001m0042,"LINESTRING (4.46454 50.84187, 4.46361 50.84327...",876.832431
1,2,1,2,CRAINHEM,ALMA,8151,8141,8151-8141,001m0042,"LINESTRING (4.45872 50.84880, 4.45865 50.84888...",514.285249
2,2,1,3,ALMA,VANDERVELDE,8141,8131,8141-8131,001m0042,"LINESTRING (4.45315 50.84988, 4.45133 50.84880...",545.002721
3,2,1,4,VANDERVELDE,ROODEBEEK,8131,8121,8131-8121,001m0042,"LINESTRING (4.44670 50.84739, 4.44541 50.84721...",799.287745
4,2,1,5,ROODEBEEK,TOMBERG,8121,8111,8121-8111,001m0042,"LINESTRING (4.43566 50.84753, 4.43476 50.84743...",827.451879


In [17]:
segments_gdf.head()

,Code_Ligne,lineId,mode,Variante,succession,start_stop_id,end_stop_id,distance_m
0,001m,1,M,1,1,8161,8151,876.83
1,001m,1,M,1,2,8151,8141,514.29
2,001m,1,M,1,3,8141,8131,545.00
3,001m,1,M,1,4,8131,8121,799.29
4,001m,1,M,1,5,8121,8111,827.45


In [10]:
# Keep a copy of the segments geodataframe
#segments_gdf.to_csv(export_folder + 'segments.csv', index=False)
# Load a copy of the segments geodataframe to avoid reprocessing each time
segments_gdf = pd.read_csv(export_folder + 'segments.csv')

##### ID-Stops Conversion
The dataset provided contains sometimes distinctions relative to the stop point (the platform, the direction, etc.) we need to be capable of converting between the ID and the stop name.

In [ ]:
# Create a conversion table between ID and generic label
IDStops = stops
IDStops['stop_id'] = IDStops['stop_id'].str[:4]
IDStops = IDStops.astype({'stop_id': 'int16'})
IDStops = IDStops[['stop_id', 'stop_name']]
IDStops.drop_duplicates(inplace=True)

In [11]:
# Keep a copy of the stops-ID conversion table
#IDStops.to_csv(export_folder + 'IDstops.csv', index=False)
# Load a copy of the stops-ID conversion table to avoid reprocessing each time
IDStops = pd.read_csv(export_folder + 'IDstops.csv')


##### Importing lines
The goal here is to import data relative to lines and their stops and process them to be used afterwards for filtering relevant stops on a line.

In [ ]:
# Load and prepare the dataframe of stops belonging to the different lines and variants
linestops_df = gpd.read_file(shapefile_stops)
linestops_df = linestops_df.to_crs(epsg=4326)

# Extract information from other columns
linestops_df['stop_id'] = linestops_df['stop_id'].str[:4]
linestops_df['lineId'] = linestops_df['Code_Ligne'].str[:3]

# Cast the type
linestops_df = linestops_df.astype({'stop_id': 'int', 'lineId': 'int'})

# Select columns and remove duplicates
linestops_df = linestops_df[["stop_id", "lineId", "mode", "Variante", "succession", "descr_fr", "geometry"]]
linestops_df = linestops_df.drop_duplicates()

In [5]:
# Keep a copy of the processed linestops dataframe
#linestops_df.to_csv(export_folder + 'linestops.csv', index=False)
# Load a copy of the processed linestops dataframe to avoid reprocessing each time
linestops_df = pd.read_csv(export_folder + 'linestops.csv')

##### Generalizing segments
This consists of several successive steps:
* Transforming IDs to stop name
* Removing unused columns
* Dropping duplicates

After these operations, we obtain a dataframe that contains segments between source and end points for a specific line variant. This will be used to obtain the distance between two stops on a line.

In [13]:
# Create a table with all the segments belonging to all the lines
reworkedSegments = segments_gdf.merge(IDStops, left_on=['start_stop_id'], right_on=['stop_id'], how='left')
reworkedSegments.rename(columns={'stop_name' : 'start_stop_name'}, inplace=True)
reworkedSegments = reworkedSegments.merge(IDStops, left_on=['end_stop_id'], right_on=['stop_id'], how='left')
reworkedSegments.rename(columns={'stop_name' : 'end_stop_name'}, inplace=True)

# Drop superseded columns 
reworkedSegments.drop(columns=['stop_id_x', 'stop_id_y', 'Code_Ligne', 'mode', 'start_stop_id', 'end_stop_id', 'succession'], inplace=True)

# Drop duplicates
reworkedSegments.drop_duplicates(['lineId', 'Variante', 'start_stop_name', 'end_stop_name'], inplace=True)

In [7]:
# Keep a copy of the reworked segments dataframe
#reworkedSegments.to_csv(export_folder + 'reworkedSegments.csv', index=False)
# Load a copy of the reworked segments dataframe to avoid reprocessing each time
reworkedSegments = pd.read_csv(export_folder + 'reworkedSegments.csv')

##### Cleaning the lines and variants
The initial dataset provides information about variants that are not part of the normal itinerary of the line (e.g. some metro 1 go to Delta at the end of their shift to go to the storage facility). As we want to use linear measures for spacial progress along a line, multiple "bastard" alternatives are not suitable to perform a successful analysis. We eliminate them from the scope of the analysis.

Required steps:
* Remove duplicate stops on the same line variant
* Reindex the succession variable
* Obtain the previous stop if existing (set the startup place otherwise)
* Obtain the distance between the two stops as determined in the previously cleaned segments dataframe
* Calculate the cumulative sum of the stop relative to the start

In [16]:
# Remove duplicates stops on the same line variant
moddedLineStops = linestops_df.drop_duplicates(['lineId', 'Variante', 'descr_fr'])

# Used to store all the line variants dataframes
tmpModded = []

# Create the list of lines
listOfLines = moddedLineStops.lineId.drop_duplicates().to_list()

for currentWorkingLine in listOfLines:
    # Get each variant of the line (normally 1/2 but you are never too cautious)
    currentLineVariants = moddedLineStops[moddedLineStops.lineId == currentWorkingLine].Variante.drop_duplicates().to_list()

    # Perform reindexing and append to tmpModded
    for currentWorkingVariant in currentLineVariants:
        myVariant = moddedLineStops[(moddedLineStops.lineId == currentWorkingLine) & (moddedLineStops.Variante == currentWorkingVariant)].sort_values(by='succession')
        variantLength = len(myVariant)
        # Change the order ID
        myVariant['succession'] = range(1, variantLength + 1)

        tmpModded.append(myVariant)

moddedLineStops = pd.concat(tmpModded).reset_index(drop=True)

# Function used to obtain the position of the previous stop on the line
def previousStopNumber(row):
    if (row['succession'] == 1):
        return row['succession']
    else:
        return row['succession'] - 1

# Obtain the position of the previous stop and their name
moddedLineStops['previous'] = moddedLineStops.apply(lambda row: previousStopNumber(row), axis=1)
# Merge to obtain the name on the line
moddedLineStops = moddedLineStops.merge(moddedLineStops[['lineId', 'Variante', 'succession', 'descr_fr']], left_on=['lineId', 'Variante', 'previous'], right_on=['lineId', 'Variante', 'succession'], how='left')
moddedLineStops.drop(columns=['previous'], inplace=True)
moddedLineStops.rename(columns={'succession_x' : 'currentPosition', 'succession_y' : 'previousPosition', 'descr_fr_x' : 'currentName', 'descr_fr_y' : 'previousName'}, inplace=True)

# Merge with the distance between the two stops
moddedLineStops = moddedLineStops.merge(reworkedSegments[['lineId', 'start_stop_name', 'end_stop_name', 'distance_m']], left_on=['lineId', 'previousName', 'currentName'], right_on=['lineId', 'start_stop_name', 'end_stop_name'], how='left')
moddedLineStops.drop(columns=['start_stop_name', 'end_stop_name'], inplace=True)
moddedLineStops['distance_m'] = moddedLineStops['distance_m'].fillna(value=0)

# Function used to obtain the position of the next stop on the line
def nextStopNumber(row):
    # Define the variant max
    maxOfLineVariant = moddedLineStops[(moddedLineStops.lineId == row['lineId']) & (moddedLineStops.Variante == row['Variante'])]['currentPosition'].max()
    # Attribute the proper number
    
    if (row['currentPosition'] == maxOfLineVariant):
        return row['currentPosition']
    else:
        return row['currentPosition'] + 1

# Obtain the position of the next stop and their name
moddedLineStops['next'] = moddedLineStops.apply(lambda row: nextStopNumber(row), axis=1)

# Merge to obtain the name on the line
moddedLineStops = moddedLineStops.merge(moddedLineStops[['lineId', 'Variante', 'currentPosition', 'currentName']], left_on=['lineId', 'Variante', 'next'], right_on=['lineId', 'Variante', 'currentPosition'], how='left')
moddedLineStops.drop(columns=['next'], inplace=True)
moddedLineStops.rename(columns={'currentPosition_x' : 'currentPosition', 'currentPosition_y' : 'nextPosition', 'currentName_x' : 'currentName', 'currentName_y' : 'nextName'}, inplace=True)

#
# Calculate the cumulated distance
#
tmpLineStops = []

# Perform on each line
for currentWorkingLine in listOfLines:
    # Get each variant of the line
    currentLineVariants = moddedLineStops[moddedLineStops.lineId == currentWorkingLine].Variante.drop_duplicates().to_list()
    
    # Perform reindexing and append to tmpModdedSegments
    for currentWorkingVariant in currentLineVariants:
        currentVariant = moddedLineStops[(moddedLineStops.lineId == currentWorkingLine) & (moddedLineStops.Variante == currentWorkingVariant)]
        currentVariant = currentVariant.sort_values(by='currentPosition').reset_index(drop=True)
        currentVariant['cumulatedDistance'] = currentVariant['distance_m'].cumsum()

        tmpLineStops.append(currentVariant)

# Merge everything under a single df
moddedLineStops = pd.concat(tmpLineStops).reset_index(drop=True)
moddedLineStops.drop(columns=['previousPosition', 'previousName'], inplace=True)

In [15]:
# Keep a copy of the modded linestops dataframe
#moddedLineStops.to_csv(export_folder + 'moddedLineStops.csv', index=False)
# Load a copy of the modded linestops dataframe to avoid reprocessing each time
moddedLineStops = pd.read_csv(export_folder + 'moddedLineStops.csv')

In [ ]:
LineStopsPosition = moddedLineStops[['lineId', 'currentName', 'currentPosition', 'Variante']].drop_duplicates(['lineId', 'currentName'])

In [16]:
# Keep a copy of the modded linestops dataframe
#LineStopsPosition.to_csv(export_folder + 'LineStopsPosition.csv', index=False)
# Load a copy of the modded linestops dataframe to avoid reprocessing each time
LineStopsPosition = pd.read_csv(export_folder + 'LineStopsPosition.csv')

##### Reworking positions
This operations aims at:
* Getting the stop names to replace the ID (and remove unknown stations) ;
* Only keep stops and terminus that actually belong to the line ;
* Define the direction of the data point ;
* Obtain the stop position on the variant.

In [22]:
# Obtain the generic label belonging to the stop ID
reworkedPositions = positions_df.merge(IDStops, left_on=['directionId'], right_on=['stop_id'], how='left')
reworkedPositions.rename(columns={'stop_name' : 'directionName'}, inplace=True)
reworkedPositions = reworkedPositions.merge(IDStops, left_on=['pointId'], right_on=['stop_id'], how='left')
reworkedPositions.rename(columns={'stop_name' : 'pointName'}, inplace=True)
reworkedPositions.drop(columns=['stop_id_x', 'stop_id_y', 'directionId', 'pointId'], inplace=True)

# Remove lines with unknown stations
reworkedPositions = reworkedPositions.dropna()

# Only keep stops that belong to the said line
reworkedPositions = reworkedPositions.merge(LineStopsPosition[['lineId', 'currentName']], left_on=['lineId', 'pointName'], right_on=['lineId', 'currentName'], how='left')
reworkedPositions = reworkedPositions.dropna(subset=['currentName'])
reworkedPositions.drop(columns=['currentName'], inplace=True)

# Only keep terminus that belong to the said line
reworkedPositions = reworkedPositions.merge(LineStopsPosition[['lineId', 'currentName']], left_on=['lineId', 'directionName'], right_on=['lineId', 'currentName'], how='left')
reworkedPositions = reworkedPositions.dropna(subset=['currentName'])
reworkedPositions.drop(columns=['currentName'], inplace=True)

# Determine the direction of the data points
reworkedPositions = reworkedPositions.merge(LineStopsPosition[['lineId', 'currentName', 'currentPosition']], left_on=['lineId', 'directionName'], right_on=['lineId', 'currentName'], how='left')
reworkedPositions.rename(columns={'currentPosition' : 'directionPosition'}, inplace=True)
reworkedPositions = reworkedPositions.merge(LineStopsPosition[['lineId', 'currentName', 'currentPosition', 'Variante']], left_on=['lineId', 'pointName'], right_on=['lineId', 'currentName'], how='left')
reworkedPositions.rename(columns={'currentPosition' : 'pointPosition'}, inplace=True)
reworkedPositions.drop(columns=['currentName_x', 'currentName_y'], inplace=True)

In [23]:
# Function used to reassign the proper variant according to the point and direction
def correctVariants(row):
    if ((row['Variante'] == 1) & (row['pointPosition'] <= row['directionPosition'])):
        return 1
    else:
        return 2

# Reassign the proper variant according to the position of the point and direction
reworkedPositions['Variante'] = reworkedPositions.apply(lambda row: correctVariants(row), axis=1)

In [24]:
# Remove previous values obtained for directionPosition and pointPosition
reworkedPositions.drop(columns=['directionPosition', 'pointPosition'], inplace=True)

# Obtain the true position of the point on the line
reworkedPositions = reworkedPositions.merge(moddedLineStops[['lineId', 'Variante', 'currentName', 'currentPosition', 'mode', 'nextName', 'cumulatedDistance']], left_on=['lineId', 'Variante', 'pointName'], right_on=['lineId', 'Variante', 'currentName'], how='left')
reworkedPositions.rename(columns={'currentPosition' : 'pointPosition'}, inplace=True)
reworkedPositions = reworkedPositions.merge(moddedLineStops[['lineId', 'Variante', 'currentName', 'currentPosition']], left_on=['lineId', 'Variante', 'directionName'], right_on=['lineId', 'Variante', 'currentName'], how='left')
reworkedPositions.rename(columns={'currentPosition' : 'directionPosition'}, inplace=True)
reworkedPositions.drop(columns=['currentName_x', 'currentName_y'], inplace=True)

# Remove problematic records and recast
reworkedPositions.dropna(subset=['directionPosition', 'pointPosition'], inplace=True)
reworkedPositions = reworkedPositions.astype({'directionPosition': 'int16', 'pointPosition': 'int16', 'Variante' : 'int16'})

In [25]:
# Keep a copy of the reworked positions dataframe
#reworkedPositions.to_csv(export_folder + 'reworkedPositions.csv', index=False)
# Load a copy of the reworked positions dataframe to avoid reprocessing each time
reworkedPositions = pd.read_csv(export_folder + 'reworkedPositions.csv')